# Chat 对象使用示例

这个示例展示了如何使用 Chat 对象进行对话管理，包括：
- 从 .env 文件加载配置
- 基本对话功能
- 异步对话
- 对话历史管理
- 错误处理
- 高级功能

## 1. 环境准备和导入模块

In [1]:
import os
from datetime import datetime

# 导入 chattool 相关模块
from chattool.core.chattype import Chat
from chattool.core.config import OpenAIConfig

## 2. 创建 Chat 对象

使用环境变量自动配置

In [2]:
# 方法1: 使用默认配置（自动从环境变量读取）
chat = Chat()
print("✅ Chat 对象创建成功（使用环境变量配置）")

# 方法2: 使用显式配置
config = OpenAIConfig(
    api_key=os.getenv('OPENAI_API_KEY'),
    api_base=os.getenv('OPENAI_API_BASE'),
    model=os.getenv('OPENAI_API_MODEL', 'gpt-3.5-turbo'),
    temperature=0.7,
    max_tokens=1000
)
chat_with_config = Chat(config=config)
print("✅ Chat 对象创建成功（使用显式配置）")

✅ Chat 对象创建成功（使用环境变量配置）
✅ Chat 对象创建成功（使用显式配置）


## 3. 基本对话示例

In [3]:
# 设置系统提示
chat.system("你是一个友善的AI助手，请用简洁明了的方式回答问题。")

# 进行第一轮对话
print("=== 第一轮对话 ===")
response1 = chat.user("你好！请简单介绍一下自己。").get_response()
print(f"用户: 你好！请简单介绍一下自己。")
print(f"助手: {response1.content}")
print(f"Token使用: {response1.total_tokens}")
print()

=== 第一轮对话 ===
用户: 你好！请简单介绍一下自己。
助手: 你好！我是一个由人工智能驱动的助手，旨在帮助回答问题、提供信息和协助解决各种任务。我可以协助你处理多种主题，无论是学习、工作还是日常生活中的问题。请随时问我任何问题！
Token使用: 94



In [4]:
# 继续对话
print("=== 第二轮对话 ===")
response2 = chat.user("能告诉我今天是星期几吗？").get_response()
print(f"用户: 能告诉我今天是星期几吗？")
print(f"助手: {response2.content}")
print(f"Token使用: {response2.total_tokens}")
print()

=== 第二轮对话 ===
用户: 能告诉我今天是星期几吗？
助手: 抱歉，我无法提供当前的日期或时间信息。您可以查看设备上的日历来找到今天是星期几。需要其他帮助吗？
Token使用: 142



## 4. 查看对话历史

In [5]:
# 查看完整对话历史
print("=== 对话历史 ===")
for i, message in enumerate(chat.chat_log):
    role = message['role']
    content = message['content'][:100] + '...' if len(message['content']) > 100 else message['content']
    print(f"{i+1}. [{role.upper()}]: {content}")

print(f"\n总消息数: {len(chat)}")
print(f"最后一条消息: {chat.last_message}")

=== 对话历史 ===
1. [SYSTEM]: 你是一个友善的AI助手，请用简洁明了的方式回答问题。
2. [USER]: 你好！请简单介绍一下自己。
3. [ASSISTANT]: 你好！我是一个由人工智能驱动的助手，旨在帮助回答问题、提供信息和协助解决各种任务。我可以协助你处理多种主题，无论是学习、工作还是日常生活中的问题。请随时问我任何问题！
4. [USER]: 能告诉我今天是星期几吗？
5. [ASSISTANT]: 抱歉，我无法提供当前的日期或时间信息。您可以查看设备上的日历来找到今天是星期几。需要其他帮助吗？

总消息数: 5
最后一条消息: 抱歉，我无法提供当前的日期或时间信息。您可以查看设备上的日历来找到今天是星期几。需要其他帮助吗？


## 5. 便捷问答方法

In [6]:
# 使用 ask 方法进行快速问答
print("=== 便捷问答 ===")
answer = chat.ask("请用一句话解释什么是人工智能？")
print(f"问题: 请用一句话解释什么是人工智能？")
print(f"回答: {answer}")

=== 便捷问答 ===
问题: 请用一句话解释什么是人工智能？
回答: 人工智能是一种能够模拟和执行人类智能任务的计算机系统或程序。


## 6. 异步对话示例

In [7]:
# 异步对话函数
async def async_chat_example():
    print("=== 异步对话示例 ===")
    
    # 创建新的对话实例
    async_chat = Chat()
    async_chat.system("你是一个编程助手，请提供简洁的编程建议。")
    
    # 异步获取响应
    response = await async_chat.ask_async("Python中如何创建虚拟环境？")
    print(f"问题: Python中如何创建虚拟环境？")
    print(f"回答: {response}")
    
    return async_chat

# 运行异步示例
async_chat = await async_chat_example()

=== 异步对话示例 ===
问题: Python中如何创建虚拟环境？
回答: 在 Python 中创建虚拟环境可以使用以下步骤：

1. **安装 `venv`**: 确保你的 Python 版本是 3.3 或更高，因为 `venv` 模块在这些版本中是自带的。如果你使用 Python 2，需要安装 `virtualenv`。

2. **创建虚拟环境**:
   - 打开终端（MacOS/Linux）或命令提示符（Windows）。
   - 导航到你希望创建虚拟环境的项目目录。
   - 运行以下命令来创建虚拟环境（假设你想创建一个名为 `venv` 的虚拟环境）：
     ```
     python -m venv venv
     ```

3. **激活虚拟环境**:
   - 在 MacOS/Linux 上，使用以下命令：
     ```bash
     source venv/bin/activate
     ```
   - 在 Windows 上，使用以下命令：
     ```cmd
     .\venv\Scripts\activate
     ```

激活后，你的命令提示符会显示虚拟环境的名称，表示你已经在虚拟环境中。

4. **安装包**:
   - 在激活的虚拟环境内，你可以使用 `pip` 来安装需要的 Python 包。例如：
     ```
     pip install requests
     ```

5. **退出虚拟环境**:
   - 在任何平台上，你可以使用以下命令退出虚拟环境：
     ```bash
     deactivate
     ```

这些步骤将帮助你在项目级别管理不同的 Python 包和版本。


## 7. 批量异步处理

In [ ]:
# 批量处理多个问题
async def batch_questions():
    print("=== 批量异步处理 ===")
    
    questions = [
        "什么是机器学习？",
        "什么是深度学习？",
        "什么是神经网络？"
    ]
    
    # 为每个问题创建独立的对话
    tasks = []
    for question in questions:
        temp_chat = Chat()
        temp_chat.system("请用一句话简洁回答问题。")
        task = temp_chat.ask_async(question)
        tasks.append((question, task))
    
    # 并发执行所有任务
    for i, (question, task) in enumerate(tasks):
        answer = await task
        print(f"{i+1}. 问题: {question}")
        print(f"   回答: {answer}")
        print()

await batch_questions()

## 8. 对话管理功能

In [ ]:
# 对话复制
print("=== 对话管理 ===")
copied_chat = chat.copy()
print(f"原对话消息数: {len(chat)}")
print(f"复制对话消息数: {len(copied_chat)}")

# 在复制的对话中尝试不同的问题
copied_response = copied_chat.ask("换个话题，你能推荐一本好书吗？")
print(f"\n复制对话中的新回答: {copied_response}")

# 删除最后一条消息
removed_message = chat.pop()
print(f"\n删除的消息: {removed_message}")
print(f"删除后消息数: {len(chat)}")

## 9. 调试和监控

In [8]:
# 调试信息
print("=== 调试和监控 ===")

# 打印对话历史
print("对话历史:")
chat.print_log()

# 获取最后一次响应的详细信息
if chat.last_response:
    print("\n最后一次响应详情:")
    print(f"ID: {chat.last_response.id}")
    print(f"模型: {chat.last_response.model}")
    print(f"创建时间: {chat.last_response.created}")
    print(f"完成原因: {chat.last_response.finish_reason}")
    print(f"Token统计: {chat.last_response.usage}")

=== 调试和监控 ===
对话历史:

--------------------------------------------------
SYSTEM
--------------------------------------------------
你是一个友善的AI助手，请用简洁明了的方式回答问题。

--------------------------------------------------
USER
--------------------------------------------------
你好！请简单介绍一下自己。

--------------------------------------------------
ASSISTANT
--------------------------------------------------
你好！我是一个由人工智能驱动的助手，旨在帮助回答问题、提供信息和协助解决各种任务。我可以协助你处理多种主题，无论是学习、工作还是日常生活中的问题。请随时问我任何问题！

--------------------------------------------------
USER
--------------------------------------------------
能告诉我今天是星期几吗？

--------------------------------------------------
ASSISTANT
--------------------------------------------------
抱歉，我无法提供当前的日期或时间信息。您可以查看设备上的日历来找到今天是星期几。需要其他帮助吗？

--------------------------------------------------
USER
--------------------------------------------------
请用一句话解释什么是人工智能？

--------------------------------------------------
ASSISTANT
--------------------------------------------------
人工智能

# 10. 异步对话示例

In [2]:
import asyncio

chat = Chat().user('请详细介绍一下Python编程语言')
async def stream_example():
    # 使用异步流式响应
    accumulated_content = ""
    
    async for response in chat.get_response_stream_async():
        # 获取增量内容
        if response.delta_content:
            accumulated_content += response.delta_content
            print(response.delta_content, end='', flush=True)
        
        # 检查是否完成
        if response.finish_reason == 'stop':
            print("\n\n流式响应完成!")
            break
    
    print(f"完整内容长度: {len(accumulated_content)}")

# 运行异步函数
asyncio.run(stream_example())


Python是一种高级编程语言，由Guido van Rossum于1980年代末创立，并于1991年首次发布。它以其简单易读的语法和广泛的应用范围而闻名，是目前最受欢迎的编程语言之一。以下是Python的一些关键特性和应用领域：

### 关键特性：
1. **简洁易读**：Python的语法设计追求代码的简洁和可读性，使得程序员能够更直观地表达逻辑，从而减少编程和维护的工作量。
   
2. **动态类型**：Python是动态类型语言，变量在第一次赋值时即被定义类型，无需显式声明类型。这提高了灵活性，但在大型项目中可能增加类型错误的风险。

3. **丰富的标准库**：Python拥有非常丰富的标准库，涵盖了文字行操作、正则表达式、文件I/O、网络编程、互联网协议、单元测试等多个领域。

4. **跨平台**：Python是跨平台的，即可以运行在Windows、macOS、Linux等各种操作系统上，只要安装了Python解释器。

5. **多范式支持**：Python支持多种编程范式，包括面向对象编程、函数式编程以及过程式编程。

6. **强大的社区**：Python拥有一个活跃的开发者社区，他们贡献了大量的第三方库和框架，如NumPy、Pandas、Django、Flask等。

7. **自动垃圾回收**：Python内置的内存管理机制能够自动回收不再使用的内存，减少程序员手动管理内存的麻烦。

### 应用领域：

1. **Web开发**：Python在Web开发领域非常流行，框架如Django和Flask使开发和部署Web应用变得简单快捷。

2. **数据科学与分析**：由于其强大的数据处理和分析库，如NumPy、Pandas、Matplotlib、Seaborn，Python已成为数据科学家的首选语言。

3. **机器学习与人工智能**：Python是许多机器学习和人工智能项目中使用的主要语言，有相关的强大工具和库，如TensorFlow、Keras、PyTorch、Scikit-learn。

4. **自动化脚本**：Python常被用于编写自动化脚本，以执行重复枯燥的任务，提高效率。

5. **学术研究与教育**：Python的简单性使其成为教学编程和研究实验的理想选择，许多高校和在线课程都采用Python作为入门语言。

6. 